# Representing Orientation in 3D

## 3D Rotation

### Right Hand Rule
![](images/Selection_097.png)
### Rotation About Axis
![](images/Selection_100.png)

**Rotation Axis: The axis that will not change**

### Example 
![](images/Selection_098.png)

### Determine rotation axis and angle for this (a is the reference coordinates) 
![](images/Selection_099.png)


### Solution

In [10]:
# Uncomment next line if pytransform3d is not installed  
# !pip install pytransform3d

In [11]:
%matplotlib qt
# %matplotlib inline

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *
X_AXIS = 0
Y_AXIS = 1
Z_AXIS = 2

In [62]:
def rotate_coordinates(axis, angle):
    p = np.array([1.0, 1.0, 1.0])
    euler = [0, 0, 0]
    vector = [0 ,0 , 0]
    vector[axis] = 1
    euler[axis] = -angle
    R = matrix_from_euler_xyz(euler)
    ax = plot_basis(R=np.eye(3), ax_s=2)
    plot_basis(ax, R, p)
    plot_axis_angle(ax,  np.hstack((vector, (angle,))), p, s=0.5)
    plt.show()
    return R

In [63]:
d  = rotate_coordinates(X_AXIS, -np.pi/2)
print(np.round(d))

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]]


In [64]:
d  = rotate_coordinates(Y_AXIS, np.pi/2)
print(np.round(d))

[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]


In [65]:
d  = rotate_coordinates(Z_AXIS, np.pi/2)
print(np.round(d))

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


## 3D Rotation Representation

### 1. Rotation Matrix Representation


$$R_x(\theta) = \begin{bmatrix}
1 & 0 & 0 \\
0 & cos(\theta) & -sin(\theta)  \\
0 & sin(\theta) & cos(\theta) \\
\end{bmatrix}$$


$$R_y(\theta) = \begin{bmatrix}
cos(\theta) & 0 & sin(\theta)  \\
0 & 1 & 0 \\
- sin(\theta) & 0 & cos(\theta)  \\
\end{bmatrix}$$


$$R_z(\theta) = \begin{bmatrix}
cos(\theta) & -sin(\theta) & 0 \\
sin(\theta) & cos(\theta) & 0 \\
0 & 0 & 1
\end{bmatrix}$$


### 2. Three-Angle Representations (Euler angles)
* Another representation of rotation angles
* For example $eurler_{xyz}$
$$
R = R_x(\phi)R_y(\theta)R_z(\psi)
$$

**Disadvantage : Gimbal Lock (Loss of singularity)**

[Source](https://www.youtube.com/watch?v=zc8b2Jo7mno)
![](images/gimbal-lock.gif)


In [146]:
# Transformation between rotation matrix and euler angles
euler_xyz = [0, 0, np.pi]
print("Eulere XYZ : \n  ", euler_xyz)
R = matrix_from_euler_xyz(euler_xyz)
print(' \n Rotation Matrix : \n', np.round(R))

Eulere XYZ : 
   [0, 0, 3.141592653589793]
 
 Rotation Matrix : 
 [[-1.  0.  0.]
 [-0. -1.  0.]
 [ 0.  0.  1.]]


In [66]:
R = np.array([[-1,  0,  0], 
              [-0, -1,  0], 
              [ 0,  0,  1]])
euler_xyz = euler_xyz_from_matrix(R)
print(euler_xyz)

[ 0.         -0.          3.14159265]


### 3. Two vectors representation

* Approach $\hat{a}$ and Orientation vector $\hat{o}$ 
* Normal vector $\hat{n} = \hat{a} \times \hat{o}$ is the cross product 
* Example (OpenGL camera coordinates) 
    * eye to center (approach0
    * up vector (orientation)
  

$$R = \begin{bmatrix}
\hat{n}_x & \hat{o}_x & \hat{a}_x \\
\hat{n}_y & \hat{o}_y & \hat{a}_y \\
\hat{n}_z & \hat{o}_z & \hat{a}_z
\end{bmatrix}$$


### 4. Rotation about arbitrary vector
* In form of $( axis[x, y, z]m, angle)$


In [100]:
axis = [0, 1, 1]
angle = np.pi/2
v = np.hstack((axis, (angle,)))
print("Axis, angle : \n ", v)
M = matrix_from_axis_angle(v)
print("\nThe Matrix : \n\n", np.round(M, 3 ))


Axis, angle : 
  [0.         1.         1.         1.57079633]

The Matrix : 

 [[ 0.    -0.707  0.707]
 [ 0.707  0.5    0.5  ]
 [-0.707  0.5    0.5  ]]


### 5. Quaternions

* Advnatages is that 
    * Easy combination
    * Easy interpolation
    
$$
\dot{q} = s + v
$$
$$
s \in R
$$
$$
v \in R^3
$$
$$
\dot{q} = s + v_1 i +  v_2 j + v_3 k 
$$
$$
i^2 = j^2 = k^2 = ijk = -1
$$

**Where in unit quaternion**
$$ s = cos(\frac{\theta}{2})$$
$$ v = sin(\frac{\theta}{2}) \hat{n}$$

In [110]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *


angle = np.pi / 2

p = np.array([1.0, 1.0, 1.0])

ax = [0, 0, 1]
a = np.hstack((ax, (angle,)))
q = quaternion_from_axis_angle(a)
print("The Quaternion Represenation : \n ",q)
R = matrix_from_quaternion(q)
ax = plot_basis(R=np.eye(3), ax_s=2)
plot_axis_angle(ax, a, p, s=0.5)
plot_basis(ax, R, p)


plt.show()

The Quaternion Represenation : 
  [0.70710678 0.         0.         0.70710678]


**Quaternions Operations**
* Quaternions product (Matrix vector multiplication)

Hamilton's quaternion multiplication. (It is not right in the reference [link](https://people.csail.mit.edu/bkph/articles/Quaternions.pdf))

$$
\hat{q}_1 \oplus \hat{q}_2 = s_1 s_2 - v_1 . v_2 , <s_1v_2 + s_2v_1+v_1\times v_2>
$$

$$
\hat{q}_1 \oplus \hat{q}_2 = \begin{bmatrix}
s & -v_1 & -v_2 & -v_3 \\
v_1 & s & -v_3 & v_2 \\
v_2 & v_3 & s & -v_1 \\
v_3 & -v_2 & v_1 & s 
\end{bmatrix} \begin{bmatrix}
s' \\
v_1' \\
v_2'\\
v_3'
\end{bmatrix}
$$

* Inversion 
$$
\ominus \dot{q} = s, <-v>
$$

In [136]:
# Example Multiplication
s, v1, v2, v3 = q[0], q[1], q[2], q[3]
q_matrix = np.array([[s,  -v1,  -v2, -v3],
                     [v1, s, -v3, v2],
                     [v2, v3,  s, -v1],
                     [v3, -v2, v1, s]])
q_square = q_matrix.dot(q)
print(np.round(q_square, 3))

[0. 0. 0. 1.]


In [137]:
new_q = concatenate_quaternions(q,q)
print(np.round(new_q, 3))

[0. 0. 0. 1.]


In [145]:
# Example Inversion
q_inv = np.hstack((q[0], (-q[1:])))
print(np.round(q_inv, 3))

[ 0.707 -0.    -0.    -0.707]


### Comparison Between different 3D Rotation Representations 

| Representation | Combine Transformations | Interpolation | AVoid Loss Singularity (Gimbal lock)|
|-------|----------------|--------------| ------|
|Rotation Matrix|$\checkmark$  | $\times$ |   $\checkmark$  |              
|Euler | $\times$  | $\checkmark$ |   $\times$  |    
|Two vectors|  $\times$  | $\checkmark$ |   $\checkmark$  |    
|Vector and angle| $\times$  | $\checkmark$ |   $\checkmark$  |    
|**Quaternion** | $\checkmark$  | $\checkmark$ |   $\checkmark$  |    

## Pose in 3D 
Pose in 3D is represented in position (translation) and Orientation (Quaternion rotation)

$$
\xi = (t, \dot{q})
$$

Or Using transformation matrix 

$$\xi = T_{4 \times 4} = \begin{bmatrix}
R_{3 \times 3}& t_{3 \times 1}\\
0_{3 \times 1}  & 1
\end{bmatrix}$$

**Same Algebric Rules**

$$\xi \oplus  0 = \xi$$
$$\xi  \ominus  0 = \xi$$
$$ \ominus X_{\xi_Y} = Y_{\xi_X}$$
$$  X_{\xi_Y} \oplus Y_{\xi_Z}  = X_{\xi_Z}$$
$$  \xi_1 \oplus \xi_2  \neq \xi_2 \oplus \xi_1$$
$$ X_p = X_{\xi_Y} . Y_p$$

## Helper functions

In [154]:
from IPython.core.debugger import set_trace

In [292]:
def get_pose_matrix(euler, p, lim = 2, s = 1, show = True):
    # Euler uses left hand but we use right hand
    euler = -1*np.array(euler)
    R = matrix_from_euler_xyz(euler)
#     set_trace()
    T = np.hstack((R, np.reshape(p,(3,1))))
    T = np.vstack((T, np.array([0, 0, 0, 1])))
    if show:
        ax = plot_basis(R=np.eye(3), ax_s=2, s = s)
        ax.set_xlim((-lim,lim))
        ax.set_ylim((-lim, lim))
        ax.set_zlim((-lim, lim))
        plot_basis(ax, R, p,s = s )
        plt.show()
   
    return T


## Exercise

### Ex1
For the given figure, find the pose of $A$ relative to $B$, and the pose of $B$ relative to $C$ as
homogeneous transformation matrices. Compose the two poses to get the pose of $A$
relative to $C$.

![](images/exercise/Selection_097.png)

**Solution**

1. $B_{\xi_A}$

In [293]:
p = np.array([3, 0, 0])
euler_xyz = [0, 0, 0]
angle = np.pi
euler_xyz[Z_AXIS] = angle
B2A = get_pose_matrix(euler_xyz, p)
print(np.round(B2A, 3))

[[-1. -0.  0.  3.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


2. $C_{\xi_B}$

In [274]:
p = np.array([2, 0, 0])
euler_xyz = [0, 0, 0]
x_angle = -60*np.pi/180
y_angle = -90*np.pi/180
z_angle = 90*np.pi/180
euler_xyz[X_AXIS] = x_angle
euler_xyz[Y_AXIS] = y_angle
euler_xyz[Z_AXIS] = z_angle
C2B = get_pose_matrix(euler_xyz, p)
print(np.round(C2B, 3))

[[ 0.    -0.    -1.     2.   ]
 [ 0.5   -0.866  0.     0.   ]
 [-0.866 -0.5    0.     0.   ]
 [ 0.     0.     0.     1.   ]]


3. $C_{\xi_A}$

$C_{\xi_A} = C_{\xi_B} B_{\xi_A} $


In [275]:
C2A = C2B.dot(B2A)
print(np.round(C2A, 3))

[[-0.     0.    -1.     2.   ]
 [-0.5    0.866  0.     1.5  ]
 [ 0.866  0.5    0.    -2.598]
 [ 0.     0.     0.     1.   ]]


In [276]:
tm = TransformManager()
tm.add_transform("A", "B", B2A)
tm.add_transform("B", "C", C2B)
C2A = tm.get_transform("A", "C")
# In the library He uses left hand coordinate 
# But we use right hand coord to get pose So order is reversed
# And this gives the same expected results
# Very important Note 
print(np.round(C2A, 3))
ax = tm.plot_frames_in("B", s=2)
lim = 3
ax.set_xlim((-lim,lim))
ax.set_ylim((-lim, lim))
ax.set_zlim((-lim, lim))
plt.show()

[[-0.     0.    -1.     2.   ]
 [-0.5    0.866  0.     1.5  ]
 [ 0.866  0.5    0.    -2.598]
 [ 0.     0.     0.     1.   ]]


### Ex2 
![](images/exercise/Selection_098.png)

**Solution**
$$
A_p = A_{\xi_B} B_p
$$

In [300]:
euler_xyz = [0, 0, 30 * np.pi/180]
t = [10, 5, 0]
# In Homogenous coord
Bp = [3, 7, 0, 1]
A2B = get_pose_matrix(euler_xyz, t, show=False )
Ap = A2B.dot(Bp)
print(np.round(Ap, 2))

[ 9.1  12.56  0.    1.  ]


### Ex3
![](images/exercise/Selection_099.png)

**Solution**

$$T = R_x(\phi) R_z(\theta) = \begin{bmatrix}
1 & 0 & 0 \\
0 & cos(\phi) & -sin(\phi)  \\
0 & sin(\phi) & cos(\phi) \\
\end{bmatrix} \begin{bmatrix}
cos(\theta) & -sin(\theta) & 0 \\
sin(\theta) & cos(\theta) & 0 \\
0 & 0 & 1
\end{bmatrix}$$



In [331]:
euler_zyx = [30*np.pi/-180, 0, 45*np.pi/-180]
T = matrix_from_euler_zyx(euler_zyx)
p = [1, 1, 1]
Ap = T.dot(p)
print(Ap)

[0.8660254  0.5        1.41421356]


### Ex4
![](images/exercise/Selection_100.png)

**Solution**
$$A_{\xi_B} = R_x(\phi) R_z(\theta) = \begin{bmatrix}
1 & 0 & 0 \\
0 & cos(\phi) & -sin(\phi)  \\
0 & sin(\phi) & cos(\phi) \\
\end{bmatrix} \begin{bmatrix}
cos(\theta) & -sin(\theta) & 0 \\
sin(\theta) & cos(\theta) & 0 \\
0 & 0 & 1
\end{bmatrix}$$

$$
A_p = A_{\xi_B} B_p
$$

So 
$$
B_p = A_{\xi_B}^{-1} A_p
$$


In [332]:
A2B = matrix_from_euler_zyx([30*np.pi/-180, 0, 45*np.pi/-180])
Ap = np.array([7, 3, 0]).T
Bp = np.linalg.inv(A2B).dot(Ap)
A2B = np.hstack((A2B,np.array([0,0,0]).reshape(3,1)))
A2B = np.vstack((A2B,np.array([0,0,0,1]).reshape(1,4)))
tm = TransformManager()
tm.add_transform("B", "A", A2B)
ax = tm.plot_frames_in("A", s=2)
print(Bp)

[ 7.56217783 -0.63775643  0.63775643]


### Ex5
![](images/exercise/Selection_101.png)

In [339]:
axis = [0.7070, 0.7070, 0]
angle = 30*np.pi/180
v = np.hstack((axis, (angle,)))
A2B = matrix_from_axis_angle(v)
print("A2B : \n", A2B)
Bp = np.array([1, 2, 0]).T
Ap = A2B.dot(Bp)
print("\nAp: \n", Ap)

A2B : 
 [[ 0.9330127   0.0669873   0.35355339]
 [ 0.0669873   0.9330127  -0.35355339]
 [-0.35355339  0.35355339  0.8660254 ]]

Ap: 
 [1.0669873  1.9330127  0.35355339]


### Ex6 
1. Get rotation matrix of rotation angle $\theta = 30$ around Z then around X with angle $\theta = 30$
1. Get rotation matrix of rotation angle $\theta = 30$ around X then around Z with angle $\theta = 30$

**Similar ??**

In [353]:
angle = 30*np.pi/-180
euler_zyx = [angle, 0, angle] 
euler_xyz = [angle, 0, angle]

Rzx = matrix_from_euler_zyx(euler_zyx)
Rxz = matrix_from_euler_xyz(euler_xyz)
print('Rzx : \n', Rzx)
print('\nRxz : \n', Rxz)

Rzx : 
 [[ 0.8660254 -0.4330127  0.25     ]
 [ 0.5        0.75      -0.4330127]
 [ 0.         0.5        0.8660254]]

Rxz : 
 [[ 0.8660254 -0.5        0.       ]
 [ 0.4330127  0.75      -0.5      ]
 [ 0.25       0.4330127  0.8660254]]


In [357]:
ax = plot_basis(R=Rxz, ax_s=2, s = 2)
plot_basis(ax, Rzx, s = 2)

### Ex7
For the given figure, find the pose of $A$ relative to $B$, and the pose of $B$ relative to $C$ as
homogeneous transformation matrices. Compose the two poses to get the pose of $A$
relative to $C$.

![](images/exercise/Selection_103.png)

**Solution**

1. $B_{\xi_A}$

In [358]:
p = np.array([0, 2, 4])
angle_x = np.pi/2
angle_y = 0
angle_z = np.pi
euler_xyz = [angle_x, angle_y, angle_z]
B2A = get_pose_matrix(euler_xyz, p)
print(np.round(B2A, 3))

[[-1. -0.  0.  0.]
 [ 0. -0. -1.  2.]
 [ 0. -1.  0.  4.]
 [ 0.  0.  0.  1.]]


2. $C_{\xi_B}$

It is simpler to get $B_{\xi_C}$ and $C_{\xi_B} =  B_{\xi_C}^{-1}$ 

In [376]:
p = np.array([3, 0, 0])
x_angle = -90*np.pi/180
y_angle = 0
z_angle = (180+36.9)*np.pi/-180
euler_xyz[X_AXIS] = x_angle
euler_xyz[Y_AXIS] = y_angle
euler_xyz[Z_AXIS] = z_angle
B2C = get_pose_matrix(euler_xyz, p, s = 3)
C2B = np.linalg.inv(B2C)
print(np.round(C2B, 3))

[[-0.8    0.    -0.6    2.399]
 [-0.6   -0.     0.8    1.801]
 [ 0.     1.     0.    -0.   ]
 [ 0.     0.     0.     1.   ]]


3. $C_{\xi_A}$

$C_{\xi_A} = C_{\xi_B} B_{\xi_A} $


In [374]:
C2A = C2B.dot(B2A)
print(np.round(C2A, 2))

[[ 0.8  0.6 -0.  -0. ]
 [ 0.6 -0.8  0.   5. ]
 [ 0.  -0.  -1.   2. ]
 [ 0.   0.   0.   1. ]]


In [380]:
tm = TransformManager()
tm.add_transform("A", "B", B2A)
tm.add_transform("B", "C", C2B)
C2A = tm.get_transform("A", "C")
# In the library He uses left hand coordinate 
# But we use right hand coord to get pose So order is reversed
# And this gives the same expected results
# Very important Note 
print(np.round(C2A, 2))
ax = tm.plot_frames_in("B", s=3)
lim = 4
ax.set_xlim((-lim,lim))
ax.set_ylim((-lim, lim))
ax.set_zlim((-lim, lim))
plt.show()

[[ 0.8  0.6 -0.  -0. ]
 [ 0.6 -0.8  0.   5. ]
 [ 0.  -0.  -1.   2. ]
 [ 0.   0.   0.   1. ]]
